In [ ]:
%matplotlib inline

In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# PHYS 395 - week 6

**Matt Wiens - #301294492**

This notebook will be organized similarly to the lab script, with major headings corresponding to the headings on the lab script.

*The TA's name (Ignacio) will be shortened to "IC" whenever used.*

## Setup

In [ ]:
# Set default plot size
plt.rcParams["figure.figsize"] = (12, 9)

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999

# Partial differential equations

## Solving Laplace's equation

Consider Laplace's equation in the form $V_{xx} + V_{yy} = 0$, where $V(x, y)$ is a potential. This is something we want to solve numerically. We can do so by using a grid with step size $h$.

On this grid, use the notation $V(x_i, y_j) = V_{i, j}$. A finite difference formula for the second derivative is

\begin{equation}
    V_{xx} = \frac{V_{i - 1, j} - 2 V_{i, j} + V_{i + 1, j}}{h^2},
\end{equation}

and similarly for $V_{yy}$.

We can come up with a nice equation for the potential at a grid point $(i, j)$ as follows:

\begin{align}
    &V_{xx} + V_{yy} = 0 \\
    &\Rightarrow 
        \frac{V_{i - 1, j} - 2 V_{i, j} + V_{i + 1, j}}{h^2}
        + \frac{V_{i, j - 1} - 2 V_{i, j} + V_{i, j + 1}}{h^2}
        = 0 \\
    &\Rightarrow V_{i, j} = \frac{1}{4} \left(
        V_{i - 1, j} + V_{i + 1, j} + V_{i, j - 1} + V_{i, j + 1}
        \right)
    .
\end{align}

See the discussion in the lab script for more details on numerical methods for solving Laplace's equation.

### Potential with fixed boundaries

Consider Laplace's equation in a square box, 1m on each side, with voltage 1V along the bottom and far wall and 0V along the others.

In [ ]:
# Set up the box matrix
box_mat = np.zeros((101, 101))

# Set up boundary conditions
box_mat[:, -1] = np.ones(101)
box_mat[-1, :] = np.ones(101)

# Tolerance
delta_tol = 1e-4

#### Jacobi update formula

Here we will solve for the potential numerically using the Jacobi update formula.

In [ ]:
iters = 0
max_diff_sqrd = 1

while max_diff_sqrd > delta_tol ** 2:
    # Increment num iteration counter
    iters += 1

    # Run Jacobi update formula
    old_mat = box_mat.copy()

    for i, j in itertools.product(range(1, 100), range(1, 100)):
        box_mat[i, j] = (
            old_mat[i - 1, j]
            + old_mat[i + 1, j]
            + old_mat[i, j - 1]
            + old_mat[i, j + 1]
        ) / 4

    # Find maximum difference
    max_diff_sqrd = ((box_mat - old_mat) ** 2).max()

This took the following number of iterations:

In [ ]:
print(iters)

Now we'll show a heatmap of the solution we computed.

In [ ]:
plt.figure()

plt.imshow(box_mat, interpolation=None)
plt.colorbar();

#### Gauss-Seidel method

In [ ]:
# Set up the box matrix
box_mat = np.zeros((101, 101))

# Set up boundary conditions
box_mat[:, -1] = np.ones(101)
box_mat[-1, :] = np.ones(101)

# Tolerance
delta_tol = 1e-4

Now we'll try using the Gauss-Seidel method using the same tolerance as before.

In [ ]:
iters = 0
max_diff_sqrd = 1

while max_diff_sqrd > delta_tol ** 2:
    # Increment num iteration counter
    iters += 1

    # Run Gauss-Seidel update formula
    old_mat = box_mat.copy()

    for i, j in itertools.product(range(1, 100), range(1, 100)):
        box_mat[i, j] = (
            box_mat[i - 1, j]
            + box_mat[i + 1, j]
            + box_mat[i, j - 1]
            + box_mat[i, j + 1]
        ) / 4

    # Find maximum difference
    max_diff_sqrd = ((box_mat - old_mat) ** 2).max()

In [ ]:
print(iters)

Now we'll show a heatmap of the solution we computed.

In [ ]:
plt.figure()

plt.imshow(box_mat, interpolation=None)
plt.colorbar();

For this method, the Gauss-Seidel used 74% of the iterations used by the Jacobi solution, and appears to give a similar solution.

#### Gauss-Seidel with SOR method

Finally, let's try the SOR method using $\omega = 1.2$.

In [ ]:
# Set up the box matrix
box_mat = np.zeros((101, 101))

# Set up boundary conditions
box_mat[:, -1] = np.ones(101)
box_mat[-1, :] = np.ones(101)

# Tolerance
delta_tol = 1e-4

In [ ]:
iters = 0
max_diff_sqrd = 1

# SOR weight
weight = 1.2

while max_diff_sqrd > delta_tol ** 2:
    # Increment num iteration counter
    iters += 1

    # Run Gauss-Seidel update formula
    old_mat = box_mat.copy()

    for i, j in itertools.product(range(1, 100), range(1, 100)):
        box_mat[i, j] = (
            box_mat[i - 1, j]
            + box_mat[i + 1, j]
            + box_mat[i, j - 1]
            + box_mat[i, j + 1]
        ) / 4

    # Apply SOR method
    box_mat = (1 - weight) * old_mat + weight * box_mat

    # Find maximum difference
    max_diff_sqrd = ((box_mat - old_mat) ** 2).max()

In [ ]:
print(iters)

Now we'll show a heatmap of the solution we computed.

In [ ]:
plt.figure()

plt.imshow(box_mat, interpolation=None)
plt.colorbar();

This method gave us a slight speed up for this example. Nothing dramatic.

### Potential of a parallel plate capacitor

DO THIS LATER

## Solving the heat/diffusion equation

Similarly to Laplace's equation we can discretize the heat equation to derive the FTCS equation, which provides a numerical method that we can use to solve the heat equation.

### Problems

#### Thermal diffusion through Earth

First, using the formulas provided in the lab script, we will plot the seasonal variation of the ground temperature in Canada's arctic.

In [ ]:
# Constants
A = -12  # deg C
B = 20  # deg C
phi = -1.9  # rads

# Get data
days = np.arange(1, 366, 1)
ground_Ts = A + B * np.sin(2 * np.pi * days / 365 + phi)

In [ ]:
# Setup figure
_, ax = plt.subplots()

plt.plot(days, ground_Ts)

# Labels
ax.set_xlabel(r"$t$ (days)")
ax.set_ylabel(r"ground $T$ ($^\circ C$)");

Now let's consider the heat equation on the domain $[0, \infty) \times [0, L]$ (where the first term in the Cartesian product is the temporal part and the second term is the spacial part), with boundary conditions

\begin{align}
    T(0, x) &= A, \\
    T(t, 0) &= A + B \sin \left( \frac{2 \pi t}{\tau} + \phi \right), \\
    T(t, L) &= A + m_T L.
\end{align}

We will solve the heat equation over 10 years numerically.

In [ ]:
# Add in extra constants
alpha = 0.1  # m^2 / day
m_T = 30e-3  # deg C / m
L = 40  # m

points_per_year = 12
num_years = 10
num_spatial_pts = 101

Before proceeding, let's verify that our stability condition is satisfied. Recall that we need to have

\begin{equation}
    \Delta t < \frac{h^2}{2 \alpha}
    .
\end{equation}

In [ ]:
print(points_per_year / 365 < (L / (num_spatial_pts - 1)) ** 2 / (2 * alpha))

Now that we've verified that our solution will be stable, we can go ahead and get solve numerically.

In [ ]:
# Set up initial guess
initial_mat = np.zeros((points_per_year * num_years, num_spatial_pts))